In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("CyberSecurityThreatDetection").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/26 12:24:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/05/26 12:24:28 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/05/26 12:24:28 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [2]:
import pandas as pd
df = pd.read_csv('preprocessed_dataset.csv')
df.head()

,Dst Port,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,Fwd Pkt Len Std,...,Idle Mean,Idle Std,Idle Max,Idle Min,Label,Src IP,Src Port,Dst IP,hour_of_day,day_of_week
0,80,10278,2,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,DDOS attack-HOIC,1,0.0,0,2,2
1,22,18,1,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,SSH-Bruteforce,1,0.0,0,3,2
2,22,6,1,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,SSH-Bruteforce,1,0.0,0,2,2
3,53,101638,1,1,40.0,146.0,40.0,40.0,40.0,0.0,...,0.0,0.0,0.0,0.0,Infilteration,1,0.0,0,11,2
4,80,119927451,119775,0,3832800.0,0.0,32.0,32.0,32.0,0.0,...,0.0,0.0,0.0,0.0,DDOS attack-LOIC-UDP,1,0.0,0,10,2


In [4]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu
!pip install pytorch-tabnet

Looking in indexes: https://download.pytorch.org/whl/cpu
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.2/536.2 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.8/175.8 MB 6.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 8.4 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 7.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 6.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 4.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 6.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9/9 [torchaudio]9 [torchaudio]]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [pytorch-tabnet]


In [5]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 1.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 603.9/603.9 kB 2.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6/6 [optuna]2m5/6 [optuna]]my]


In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from imblearn.under_sampling import RandomUnderSampler
import torch
from pytorch_tabnet.tab_model import TabNetClassifier
import optuna
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score 

/home/CholePuri/cyberenv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
# Separate features (X) and target (y)
X = df.drop('Label', axis=1)
y = df['Label']

# Encode the target variable
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Handle class imbalance using RandomUnderSampler
# This will reduce the number of samples in the majority class(es)
rus = RandomUnderSampler(random_state=42)
X_resampled, y_resampled = rus.fit_resample(X, y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42, stratify=y_resampled)

# Scale numerical features
# Identify numerical columns after encoding categorical ones
numerical_cols = X_train.select_dtypes(include=['int64', 'float64']).columns

# Before scaling, check for and handle infinite values in numerical columns
# Replace infinite values with a large finite number or the maximum value of the column
# For simplicity, let's replace them with NaN first, then the existing fillna logic will handle them.
# Or directly replace with a finite value, 0 or a calculated value.

import numpy as np

# Check for infinite values in the numerical columns of X_train
for col in numerical_cols:
    if np.isinf(X_train[col]).any():
        print(f"Column '{col}' in X_train contains infinite values. Replacing with 0.")
        # Replace infinite values with a finite number, e.g., 0
        X_train[col].replace([np.inf, -np.inf], 0, inplace=True) # You could also use X_train[col].max() if suitable

# Check for infinite values in the numerical columns of X_test as well, although the error was in train
# It's good practice to ensure consistency
for col in numerical_cols:
    if np.isinf(X_test[col]).any():
        print(f"Column '{col}' in X_test contains infinite values. Replacing with 0.")
        X_test[col].replace([np.inf, -np.inf], 0, inplace=True)


scaler = StandardScaler()
X_train[numerical_cols] = scaler.fit_transform(X_train[numerical_cols])
X_test[numerical_cols] = scaler.transform(X_test[numerical_cols])

Column 'Flow Pkts/s' in X_train contains infinite values. Replacing with 0.


/tmp/ipykernel_13254/2518751553.py:33: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_train[col].replace([np.inf, -np.inf], 0, inplace=True) # You could also use X_train[col].max() if suitable


In [10]:
# Define the objective function for Optuna
def objective(trial):
    # Define the hyperparameter search space
    # TabNet specific hyperparameters
    n_d_a = trial.suggest_int('n_d_a', 8, 64) # Common size for attention and decision layers
    n_steps = trial.suggest_int('n_steps', 3, 10)
    gamma = trial.suggest_float('gamma', 1.0, 2.0) # Higher gamma means more importance to previous step's attention
    lambda_sparse = trial.suggest_float('lambda_sparse', 1e-6, 1e-3, log=True) # Controls sparsity
    mask_type = trial.suggest_categorical('mask_type', ['sparsemax', 'entmax'])

    # Optimizer hyperparameters
    lr = trial.suggest_float('lr', 1e-5, 1e-2, log=True)

    # Training parameters (can also be tuned, but start with model params)
    max_epochs = 100 # Reduce epochs for faster tuning, then train with more
    patience = 20 # Patience for early stopping
    batch_size = trial.suggest_categorical('batch_size', [512, 1024, 2048])
    virtual_batch_size = trial.suggest_categorical('virtual_batch_size', [64, 128, 256])


    # Create and train the TabNet model with suggested hyperparameters
    model = TabNetClassifier(
        n_d=n_d_a,
        n_a=n_d_a,
        n_steps=n_steps,
        gamma=gamma,
        lambda_sparse=lambda_sparse,
        mask_type=mask_type,
        optimizer_fn=torch.optim.Adam,
        optimizer_params=dict(lr=lr),
        scheduler_params={"step_size":20, "gamma":0.9}, # Keep scheduler simple for now
        scheduler_fn=torch.optim.lr_scheduler.StepLR,
        verbose=0 # Suppress verbose output during tuning
    )

    # Convert data to numpy arrays (already done in the original code block, but good to be explicit in the function)
    # Ensure X_train_np, y_train_np, X_test_np, y_test_np are accessible in this scope
    # Since this objective function will likely be run within the same notebook session,
    # the global variables defined in the previous cell should be available.

    try:
        model.fit(
            X_train=X_train_np, y_train=y_train_np,
            eval_set=[(X_test_np, y_test_np)],
            eval_name=['test'],
            max_epochs=max_epochs,
            patience=patience,
            batch_size=batch_size,
            virtual_batch_size=virtual_batch_size,
            num_workers=0,
            drop_last=False
        )

        # Evaluate the model on the test set
        preds = model.predict(X_test_np)

        # Calculate the metric to optimize. Accuracy is simple, but for imbalanced data,
        # F1-score or ROC AUC might be better. Let's use accuracy for now as requested,
        # but keep in mind other metrics.
        accuracy = accuracy_score(y_test_np, preds)

        # You could also return other metrics:
        # f1 = f1_score(y_test_np, preds, average='weighted') # or 'macro', 'micro'
        # roc_auc = roc_auc_score(y_test_np, model.predict_proba(X_test_np), multi_class='ovr') # ovr or ovo

        return accuracy

    except Exception as e:
        # Handle potential errors during training (e.g., CUDA out of memory)
        # Returning a very low value discourages Optuna from exploring this configuration
        print(f"Trial failed with error: {e}")
        return 0.0 # Or np.nan, depending on Optuna version/settings

# Create an Optuna study
study = optuna.create_study(direction='maximize') # We want to maximize accuracy

# Run the optimization
# n_trials: number of different hyperparameter combinations to try
# timeout: maximum time in seconds to run the study
study.optimize(objective, n_trials=50, timeout=600) # Run 50 trials, max 10 minutes

# Print the best hyperparameters and the corresponding best score
print("Best trial:")
trial = study.best_trial

print(f"  Value: {trial.value}")
print("  Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

# You can access the best model parameters from study.best_params and
# train your final model using these parameters on the full training data (or train + validation data if you split differently).

# Train the final model with the best hyperparameters
best_params = study.best_params
print("\nTraining final model with best parameters:")
final_clf = TabNetClassifier(
    n_d=best_params['n_d_a'],
    n_a=best_params['n_d_a'],
    n_steps=best_params['n_steps'],
    gamma=best_params['gamma'],
    lambda_sparse=best_params['lambda_sparse'],
    mask_type=best_params['mask_type'],
    optimizer_fn=torch.optim.Adam,
    optimizer_params=dict(lr=best_params['lr']),
    scheduler_params={"step_size":50, "gamma":0.9}, # Use the original scheduler or tune this too
    scheduler_fn=torch.optim.lr_scheduler.StepLR,
    verbose=1 # Keep verbose for final training
)

import matplotlib.pyplot as plt

# Train the final model with more epochs
final_clf.fit(
    X_train=X_train_np, y_train=y_train_np,
    eval_set=[(X_train_np, y_train_np), (X_test_np, y_test_np)],  # Include train & test for plotting
    eval_name=['train', 'test'],
    max_epochs=300,          # Increased to 300 as requested
    patience=50,             # You can adjust this as needed
    batch_size=best_params['batch_size'],
    virtual_batch_size=best_params['virtual_batch_size'],
    num_workers=0,
    drop_last=False
)

# Evaluate the final model
final_preds = final_clf.predict(X_test_np)
final_accuracy = accuracy_score(y_test_np, final_preds)
print(f"\nFinal model accuracy with best hyperparameters: {final_accuracy}")

# ========== Plotting Train vs Validation Accuracy ==========
# Extract logs
train_acc = final_clf.history['train_accuracy']
valid_acc = final_clf.history['test_accuracy']  # 'test' is the name you assigned in eval_name

# Plot
plt.figure(figsize=(10, 5))
plt.plot(train_acc, label='Train Accuracy')
plt.plot(valid_acc, label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Train vs Validation Accuracy')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

[I 2025-05-26 12:47:27,641] A new study created in memory with name: no-name-adb3579d-6276-4e2b-b298-b1d0037eeb33
[I 2025-05-26 12:47:27,649] Trial 0 finished with value: 0.0 and parameters: {'n_d_a': 47, 'n_steps': 5, 'gamma': 1.0341542449701877, 'lambda_sparse': 1.107429994876923e-06, 'mask_type': 'entmax', 'lr': 1.946532219615179e-05, 'batch_size': 1024, 'virtual_batch_size': 64}. Best is trial 0 with value: 0.0.
[I 2025-05-26 12:47:27,654] Trial 1 finished with value: 0.0 and parameters: {'n_d_a': 43, 'n_steps': 10, 'gamma': 1.503559397067173, 'lambda_sparse': 6.24440479977491e-06, 'mask_type': 'entmax', 'lr': 0.0004667129140245494, 'batch_size': 1024, 'virtual_batch_size': 64}. Best is trial 0 with value: 0.0.
[I 2025-05-26 12:47:27,658] Trial 2 finished with value: 0.0 and parameters: {'n_d_a': 58, 'n_steps': 8, 'gamma': 1.5880422688796219, 'lambda_sparse': 7.612427678905913e-06, 'mask_type': 'entmax', 'lr': 1.0884915113712385e-05, 'batch_size': 1024, 'virtual_batch_size': 64}. B

Trial failed with error: name 'X_train_np' is not defined
Trial failed with error: name 'X_train_np' is not defined
Trial failed with error: name 'X_train_np' is not defined
Trial failed with error: name 'X_train_np' is not defined
Trial failed with error: name 'X_train_np' is not defined
Trial failed with error: name 'X_train_np' is not defined
Trial failed with error: name 'X_train_np' is not defined
Trial failed with error: name 'X_train_np' is not defined
Trial failed with error: name 'X_train_np' is not defined
Trial failed with error: name 'X_train_np' is not defined
Trial failed with error: name 'X_train_np' is not defined
Trial failed with error: name 'X_train_np' is not defined
Trial failed with error: name 'X_train_np' is not defined
Trial failed with error: name 'X_train_np' is not defined


[I 2025-05-26 12:47:27,862] Trial 14 finished with value: 0.0 and parameters: {'n_d_a': 52, 'n_steps': 9, 'gamma': 1.4397661061810965, 'lambda_sparse': 1.1200754143506816e-06, 'mask_type': 'entmax', 'lr': 0.007405168875795538, 'batch_size': 1024, 'virtual_batch_size': 64}. Best is trial 0 with value: 0.0.
[I 2025-05-26 12:47:27,890] Trial 15 finished with value: 0.0 and parameters: {'n_d_a': 37, 'n_steps': 6, 'gamma': 1.1351451531770569, 'lambda_sparse': 1.7341132883192343e-05, 'mask_type': 'entmax', 'lr': 2.364972146575874e-05, 'batch_size': 2048, 'virtual_batch_size': 64}. Best is trial 0 with value: 0.0.
[I 2025-05-26 12:47:27,928] Trial 16 finished with value: 0.0 and parameters: {'n_d_a': 63, 'n_steps': 7, 'gamma': 1.4890446209394954, 'lambda_sparse': 2.5214495016147316e-06, 'mask_type': 'entmax', 'lr': 0.00018910475332187203, 'batch_size': 1024, 'virtual_batch_size': 256}. Best is trial 0 with value: 0.0.
[I 2025-05-26 12:47:27,960] Trial 17 finished with value: 0.0 and parameter

Trial failed with error: name 'X_train_np' is not defined
Trial failed with error: name 'X_train_np' is not defined
Trial failed with error: name 'X_train_np' is not defined
Trial failed with error: name 'X_train_np' is not defined
Trial failed with error: name 'X_train_np' is not defined
Trial failed with error: name 'X_train_np' is not defined
Trial failed with error: name 'X_train_np' is not defined


[I 2025-05-26 12:47:28,079] Trial 21 finished with value: 0.0 and parameters: {'n_d_a': 59, 'n_steps': 9, 'gamma': 1.5686635347634696, 'lambda_sparse': 8.681858729997459e-06, 'mask_type': 'entmax', 'lr': 1.1214658873746498e-05, 'batch_size': 1024, 'virtual_batch_size': 64}. Best is trial 0 with value: 0.0.
[I 2025-05-26 12:47:28,110] Trial 22 finished with value: 0.0 and parameters: {'n_d_a': 10, 'n_steps': 8, 'gamma': 1.659450979872429, 'lambda_sparse': 8.42855915936485e-06, 'mask_type': 'entmax', 'lr': 2.0099795355126918e-05, 'batch_size': 1024, 'virtual_batch_size': 64}. Best is trial 0 with value: 0.0.
[I 2025-05-26 12:47:28,141] Trial 23 finished with value: 0.0 and parameters: {'n_d_a': 56, 'n_steps': 10, 'gamma': 1.4940358860854492, 'lambda_sparse': 2.4787780905514452e-05, 'mask_type': 'entmax', 'lr': 4.7464206011778686e-05, 'batch_size': 1024, 'virtual_batch_size': 64}. Best is trial 0 with value: 0.0.
[I 2025-05-26 12:47:28,169] Trial 24 finished with value: 0.0 and parameters

...

[I 2025-05-26 12:47:28,526] Trial 36 finished with value: 0.0 and parameters: {'n_d_a': 37, 'n_steps': 4, 'gamma': 1.4439701069078255, 'lambda_sparse': 2.0306315520362545e-05, 'mask_type': 'entmax', 'lr': 0.0006098084640255853, 'batch_size': 512, 'virtual_batch_size': 64}. Best is trial 0 with value: 0.0.
[I 2025-05-26 12:47:28,552] Trial 37 finished with value: 0.0 and parameters: {'n_d_a': 23, 'n_steps': 6, 'gamma': 1.6139649149136497, 'lambda_sparse': 8.18917594182019e-05, 'mask_type': 'sparsemax', 'lr': 0.0032715770558118203, 'batch_size': 1024, 'virtual_batch_size': 128}. Best is trial 0 with value: 0.0.
[I 2025-05-26 12:47:28,573] Trial 38 finished with value: 0.0 and parameters: {'n_d_a': 19, 'n_steps': 10, 'gamma': 1.1967344112132396, 'lambda_sparse': 1.0794869596716814e-05, 'mask_type': 'entmax', 'lr': 0.0002870694613390964, 'batch_size': 2048, 'virtual_batch_size': 64}. Best is trial 0 with value: 0.0.
[I 2025-05-26 12:47:28,598] Trial 39 finished with value: 0.0 and paramete

Trial failed with error: name 'X_train_np' is not defined
Trial failed with error: name 'X_train_np' is not defined
Trial failed with error: name 'X_train_np' is not defined
Trial failed with error: name 'X_train_np' is not defined
Trial failed with error: name 'X_train_np' is not defined
Trial failed with error: name 'X_train_np' is not defined
Trial failed with error: name 'X_train_np' is not defined
Trial failed with error: name 'X_train_np' is not defined
Trial failed with error: name 'X_train_np' is not defined


[I 2025-05-26 12:47:28,743] Trial 44 finished with value: 0.0 and parameters: {'n_d_a': 43, 'n_steps': 9, 'gamma': 1.9151230285541057, 'lambda_sparse': 3.896003723565871e-06, 'mask_type': 'sparsemax', 'lr': 0.0010394526350858426, 'batch_size': 2048, 'virtual_batch_size': 128}. Best is trial 0 with value: 0.0.
[I 2025-05-26 12:47:28,775] Trial 45 finished with value: 0.0 and parameters: {'n_d_a': 38, 'n_steps': 8, 'gamma': 1.738479047486464, 'lambda_sparse': 6.227932189282702e-06, 'mask_type': 'sparsemax', 'lr': 0.0015983597983872352, 'batch_size': 1024, 'virtual_batch_size': 64}. Best is trial 0 with value: 0.0.
[I 2025-05-26 12:47:28,802] Trial 46 finished with value: 0.0 and parameters: {'n_d_a': 59, 'n_steps': 10, 'gamma': 1.0447723060913388, 'lambda_sparse': 2.1210757408237577e-06, 'mask_type': 'entmax', 'lr': 0.0006746110554126594, 'batch_size': 1024, 'virtual_batch_size': 128}. Best is trial 0 with value: 0.0.
[I 2025-05-26 12:47:28,828] Trial 47 finished with value: 0.0 and para

Trial failed with error: name 'X_train_np' is not defined
Trial failed with error: name 'X_train_np' is not defined
Trial failed with error: name 'X_train_np' is not defined
Trial failed with error: name 'X_train_np' is not defined
Trial failed with error: name 'X_train_np' is not defined
Best trial:
  Value: 0.0
  Params: 
    n_d_a: 47
    n_steps: 5
    gamma: 1.0341542449701877
    lambda_sparse: 1.107429994876923e-06
    mask_type: entmax
    lr: 1.946532219615179e-05
    batch_size: 1024
    virtual_batch_size: 64

Training final model with best parameters:


NameError: name 'X_train_np' is not defined